In [ ]:
"""
RAMS query program

Created on Wed Jul 5 11:49:17 2017

@author: Tingting Huang

thuang1@iastate.edu

"""

# Step 1

Use your own directory and filename.

Must have column names: routeId, MM

In [ ]:
dirpath = 'C:/Users/thuang1/Desktop/REST API/'
filename = 'MM_Found_XD_Coded_Direction_Checked.csv'

# Step2

Install arcgis package by shell command: 
pip install arcgis-rest-query

In [ ]:
import pandas as pd
import numpy as np
import json
import requests
import os
from arcgis import ArcGIS

# connect to service
username = os.getenv('ARCGIS_USERNAME', None)
password = os.getenv('ARCGIS_PASSWORD', None)
service = ArcGIS("https://gis.iowadot.gov/ramsa/rest/services/lrs/MapServer/98/query",
username=username,
password=password)

# Step 3

a. Load functions

In [ ]:
def valid_post(r, x0):

    
    
    res=[]
    if len(r['features'])==0:
        pass
    else:
        for i in xrange(len(r['features'])):
            ss=str(r['features'][i]['attributes']['REFERENCE_POST_VALUE'])
            if len([s for s in ss if s.isdigit()])==len(ss):
                res.append([float(r['features'][i]['attributes']['MEASURE']),\
                            float(r['features'][i]['attributes']['REFERENCE_POST_VALUE'])])
    
    temp = pd.DataFrame(res, columns=['MM','Post'])
    temp = temp.sort_values(by='Post').reset_index(drop=True)
   
    return temp

# updated finding schema, use two closest as x1, x2
def get_near_two(temp, x0):
    if min(temp.MM)>x0: 
        one=temp['MM'].idxmin()
        mini = temp['MM'].min()
        two=temp.loc[temp.MM>mini,'MM'].idxmin()
        
    elif max(temp.MM)<x0:
        one=temp['MM'].idxmax()
        maxi = temp['MM'].max()
        two=temp.loc[temp.MM<maxi,'MM'].idxmax()
        
    else: 
        one=temp.loc[temp.MM<x0,'MM'].idxmax()
        two=temp.loc[temp.MM>x0,'MM'].idxmin()
        
    x1, y1, x2, y2 = temp.loc[one,'MM'],  temp.loc[one,'Post'], temp.loc[two,'MM'], temp.loc[two,'Post']
    
    return x1, y1, x2, y2


def get_mile_post(df):
    
    routeId = df['routeId']
    a = df['a']
    b = df['b']
    x0 = df['MM']
    index = df.name
    
    try:

        r = service.get_json(98, where= "EFFECTIVE_END_DATE is NULL AND ROUTE_ID = '%s' AND measure >= '%s' AND measure< '%s'" % (routeId, a, b),\
                                fields=fields)
        temp = valid_post(r, x0)

        if len(temp)>=2:
            x1, y1, x2, y2 = get_near_two(temp, x0)
            
            b1=(y2-y1)/(x2-x1)
            b0=y2-b1*x2
            y0=b0+b1*x0

        else:
            r = service.get_json(98, where= "EFFECTIVE_END_DATE is NULL AND ROUTE_ID = '%s' AND measure >= '%s' AND measure< '%s'" % (routeId, a-20, b+20),\
                                fields=fields) 
            temp = valid_post(r, x0)

            if (len(temp)==0) or (len(temp)==1):
                y0=None
            else:
                x1, y1, x2, y2 = get_near_two(temp, x0)
                
                b1=(y2-y1)/(x2-x1)
                b0=y2-b1*x2
                y0=b0+b1*x0

        return y0

    except Exception, e:
        print "Error! At row "+str(index)+": "+str(e)

b. Import your data and create boundary

In [ ]:
df = pd.read_csv(dirpath+filename)

df['a']=np.floor(df.MM)-3
df['b']=np.ceil(df.MM)+3

c. Specify query fields

In [ ]:
fields=['REFERENCE_POST_VALUE','MEASURE']

d. Main function

In [ ]:
from datetime import datetime
now1=datetime.now()


df['mile_post']=df.apply(lambda x: get_mile_post(x), axis=1)


    
now2=datetime.now()

print('Runtime: '+str(now2-now1))

e. Export data

In [ ]:

df=df.drop(['a','b'],axis=1)
df.to_csv(dirpath+'Reference_Post_'+filename, header=True, index=False)
